In [ ]:
from datetime import datetime
from netCDF4 import Dataset as ds
import numpy as np
import dateutil as du 
import parser
import pytz
import dateutil.parser 
import pandas as pd
from pandas import DataFrame
import csv
import glob, os
from os.path import expanduser
from pathlib import Path
import xarray as xr 

In [ ]:
### Load .CSV and 
### subset bleaching events (conditional, e.g. exclude NaN)
### Filter only CodeID, site names and country" 
df = pd.read_csv('CBdbV2.csv',encoding='latin-1', low_memory=False)
df = df[['RECORD_ID','COUNTRY','LAT_corrected','LON_corrected','SEVERITY_CODE','DATE','MONTH','YEAR']]
df = pd.DataFrame(df)
#### Convert index into column
#df.reset_index(level=0, inplace=True)
df['ITEM_ID'] = df.index.astype(str).str.zfill(7)   # add '0s' at the beginning of the number and convert as string 
df.rename(columns={'DATE':'DAY'}, inplace = True) #use "implace = True" to rename the existing DataFrame
df = df[pd.to_numeric(df['DAY'], errors='coerce').notnull()]  # Drop rows that do not have as.numeric DATE (DAY)
df['DAY'] = df['DAY'].apply(lambda x: int(float(x)))
df['DAY'] = df['DAY'].apply('{0:0>2}'.format)
df['MONTH'] = df['MONTH'].apply(lambda x: int(float(x)))
df['MONTH'] = df['MONTH'].apply('{0:0>2}'.format)
df['full_date'] = df.apply(lambda x: '%s-%s-%s' % (x['YEAR'], int(float(x['MONTH'])), x['DAY']), axis=1) # create a column in datetime format; %s - String (or any object with a string representation, like numbers), %d - Integers %f - Floating point numbers
df['full_date'] = df['full_date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d")) # use datetime funtion to convert string 'full_date' to datetime  
df['JD'] = df['full_date'].apply(lambda x: x.timetuple().tm_yday)
### Re-order by column starting by 'ID' and renaming LAT and LON
df = df.reindex(columns=['ITEM_ID','COUNTRY','LAT_corrected','LON_corrected','SEVERITY_CODE','DAY','MONTH','YEAR','full_date','JD'])
df.rename(columns={'LAT_corrected': 'lat', 'LON_corrected': 'lon'}, inplace=True) # use "implace = True" to rename the existing DataFrame (rather than creating a copy)
df = df.dropna(axis=0, how='any')  # Drop row with NaN if any

In [ ]:
df.head()

In [ ]:
# Function to slice the filename with .nc extention, this will be used to open the files in directory
def file_info(ncfilename):
    with ds(ncfilename,'r') as ncdat:
        #string = str(ncfilename)
        #filename = string[-16:] 
        array = xr.open_dataset(ncfilename) # open_dataset works fine with either full path or filename.nc 
        return array  
def array_sel(array):
    dsloc = array.sel(lon=lon,lat=lat, method='nearest') 
    val_arr = dsloc['analysed_sst']
    val_arr = val_arr.rolling(time=90).std()
    val_arr = val_arr.sel(time=time.strftime("%Y-%m-%d")) # slice to a single given date
    return val_arr 

In [ ]:
# Extract arrays per row and put them in a list
home = Path(expanduser("~"))
x = []
for row in df.itertuples():
        lat = row.lat
        lon = row.lon
        time = row.full_date
        path = home / Path("E:/PGonzalez/SST_CRW/coraltemp_1985-2017/coraltemp_v1.0_1985-2019.nc") 
        array = file_info(path)
        cf = array_sel(array)
        x.append(cf)

In [ ]:
# # create a list for sst values # create lists for coordinates (time) 
std_90 = [item.values for item in x]   # get array of values; analysed sst in this case
df['std_90'] = std_90
df.to_csv('coraltemp_runningstd90_dayofreport.csv')

In [ ]:
# # create a list for sst values # create lists for coordinates (time) 
# sst = [item[0].values for item in z]   # get array of values; analysed sst in this case
# rm = [item[0].values for item in rmonth]  # get the rank of the month according to the warmest month of the climatology
# rmx = [item[0].values for item in rmax] # get the warmest month of the climatology 

# coor = [item[0].coords['time'] for item in z]    # get array of values of coordinate 'time' for analysed sst

In [ ]:
# # create lists of values (actual time) of coordinates  
# coor_list = []
# rm_list = []
# rmx_list = []
# for item in coor: 
#     cor_val = item.values
#     coor_list.append(cor_val)
# for item in rm: 
#     rm_val = item.values
#     rm_list.append(rm_val)
# for item in rmx: 
#     rmx_val = item.values
#     rmx_list.append(rmx_val)    

In [ ]:
# df_sst = pd.DataFrame(sst, columns = ['MMMmax_SST']) 
# df_coor = pd.DataFrame(coor_list, columns=['Date'])
# df_rm = pd.DataFrame(rm, columns=['rank_month_date'])
# df_rmx = pd.DataFrame(rmx, columns=['warmest_month'])
# df_rank = pd.concat([df_sst, df_coor, df_rm, df_rmx ], axis=1)
# #df.to_csv('df_MMMmax_date.csv', index=False)

In [ ]:
# rank_value = [item[0].values for item in x] 
# df_rank = pd.DataFrame(rank_value, columns = ['rank_cold_to_warm'])
# df_rank

In [ ]:
# def asc(item):
#     'This function reverse the ranking to warmest = 1 to coldest = 12'
#     if item == str(1.0):
#         return '12'
#     elif item == str(2.0):
#         return '11'  
#     elif item == str(3.0):
#         return '10'
#     elif item == str(4.0):
#         return '9'  
#     elif item == str(5.0):
#         return '8'  
#     elif item == str(6.0):
#         return '7'  
#     elif item == str(7.0):
#         return '6'  
#     elif item == str(8.0):
#         return '5'  
#     elif item == str(9.0):
#         return '4'  
#     elif item == str(10.0):
#         return '3'  
#     elif item == str(11.0):
#         return '2'  
#     elif item == str(12.0):
#         return '1'  

In [ ]:
# warm = []
# for row in df_rank.itertuples():
#     rank = row.rank_cold_to_warm
#     s = asc(str(rank))
#     warm.append(s)
# df_warm = pd.DataFrame(warm, columns=['rank_warm_to_cold'])
# df_warm_rank = pd.concat([df_rank, df_warm], axis=1)
# df_warm_rank